In [ ]:
import galois
import numpy as np
import sys
sys.path.append('../../doubling-CSST/')
from convert_alist import readAlist
alistDirPath = "../../doubling-CSST/alistMats/GO03_self_dual/"
# The following are the parameters of the self-dual code included in the alistDirPath
length_dist_dict = {4:2, 6:2, 8:2, 10:2, 12:4, 14:4, 16:4, 18:4, 20:4, 22:6, 24:6, 26:6, 28:6, 30:6, 32:8, 34:6, 36:8, 38:8, 40:8, 42:8, 44:8, 46:8, 48:8, 50:8, 52:10, 54:8, 56:10, 58:10, 60:12, 62:10, 64:10}

n = 12 # all even lengths from 4 to 64
d = length_dist_dict[n]
F2 = galois.GF(2)
alistFilePath = alistDirPath + "n" + str(n) + "_d" + str(d) + ".alist"

GenMat = F2(readAlist(alistFilePath))
print("Generator matrix of the [n=%s,k=%s,d=%s] self-dual code:" % (n, n//2, d))
print(GenMat)

Generator matrix of the [n=12,k=6,d=4] self-dual code:
[[0 1 1 1 0 0 1 1 1 0 0 0]
 [0 1 1 0 0 1 1 1 0 0 1 0]
 [1 0 0 0 0 1 1 1 0 0 1 1]
 [0 0 1 0 0 1 1 0 0 1 1 1]
 [0 0 1 1 1 0 0 0 0 1 1 1]
 [0 1 1 1 0 0 1 0 0 1 1 0]]


In [89]:
import scipy.sparse

G_punctured = GenMat[:, :-1]  # Puncture the last column
print("Punctured code: [n=%s,k=%s]" % (G_punctured.shape[1], G_punctured.shape[0]))
H_punctured = G_punctured.null_space() # Parity-check matrix of the punctured code = generator matrix of the dual of the punctured code
print("Dual of the punctured code: [n=%s,k=%s]" % (H_punctured.shape[1], H_punctured.shape[0]))
Hx = Hz = scipy.sparse.csr_matrix(np.array(H_punctured, dtype=int))

print(type(Hx))
print(Hz.size)

Punctured code: [n=11,k=6]
Dual of the punctured code: [n=11,k=5]
<class 'scipy.sparse._csr.csr_matrix'>
22


In [95]:
from ldpc.bplsd_decoder import BpLsdDecoder
bias_factor = 0.0
num_shots = 10
error_rate = 0.1
lsd_order = 5
max_iter = 2

bp_lsd_x = BpLsdDecoder(
    Hx,
    error_rate=error_rate,
    bp_method = 'product_sum',
    max_iter = max_iter,
    schedule = 'serial',
    lsd_method = 'lsd_cs',
    lsd_order = lsd_order
)

bp_lsd_z = BpLsdDecoder(
    Hz,
    error_rate=error_rate,
    bp_method = 'product_sum',
    max_iter = max_iter,
    schedule = 'serial',
    lsd_method = 'lsd_cs',
    lsd_order = lsd_order
)

rZ = bias_factor / (1 + bias_factor)
rX = rY = (1 - rZ) / 2

error_x = np.random.choice(
    [0, 1], size=(num_shots, Hx.shape[1]), p=[1 - rX * error_rate, rX * error_rate]
)
error_y = np.random.choice(
    [0, 1], size=(num_shots, Hx.shape[1]), p=[1 - rY * error_rate, rY * error_rate]
)
error_z = np.random.choice(
    [0, 1], size=(num_shots, Hx.shape[1]), p=[1 - rZ * error_rate, rZ * error_rate]
)

In [96]:
error_x = (error_x + error_y) % 2
error_z = (error_z + error_y) % 2

syndromes_x = (error_z @ Hx.T) % 2
syndromes_z = (error_x @ Hz.T) % 2


# Decode the syndromes for X and Z errors separately
for i in range(num_shots):
	print("Shot %d:" % i)	
	residual_error_x = (bp_lsd_x.decode(syndromes_x[i].T) + error_z[i]) % 2
	residual_error_z = (bp_lsd_z.decode(syndromes_z[i].T) + error_x[i]) % 2
	print(np.sum(residual_error_x + residual_error_z))


# Ensure decoding_x and error_x have the same shape before comparison
#logical_error_x = (residual_error_x @ Lx.T) % 2
#logical_error_z = (residual_error_z @ Lz.T) % 2

#np.sum(logical_error_x + logical_error_z)

Shot 0:
0
Shot 1:
0
Shot 2:
0
Shot 3:
5
Shot 4:
0
Shot 5:
0
Shot 6:
4
Shot 7:
6
Shot 8:
0
Shot 9:
0
